Importamos ibliotecas Pertinentes

In [27]:
import pandas as pd
from openpyxl import load_workbook
import numpy as np

Cargamos archivos para crear los DataFrames

In [28]:
negocio_bd_GR8 = pd.read_excel('bd_negocio_GR8.xlsx')
bd_negocio_DISTR = pd.read_excel('bd_negocio_DISTR.xlsx')
bd_negocio_DELATSA = pd.read_excel('bd_negocio_DELATSA.xlsx')

bd_1 = 'bd_competencia.xlsx'
compe_1 = pd.read_excel(bd_1, sheet_name = 'dia_1')
compe_2 = pd.read_excel(bd_1, sheet_name = 'dia_2')
ref = pd.read_excel('referencias.xlsx')
prom = pd.read_excel('costo_promedio.xlsx.')


In [29]:
# Parámetros de Mercado Libre
iva = 1.16
isr = 1.13
envio = 90

Transformaciones y Limpieza

Comenzamos con el DataFrame de las Bases de Datos del negocio

In [30]:
#Transformacion
#Renombrar la columnas para que coincida con el nombre de las columnas del archivo de bd_competencia
negocio_bd_GR8.rename(columns={"ID": "Id", "TipoPublicacion(Clasica,Premium)": "Tipo de Publicación", "Att_SellerSKU": "SKU", "NumeroParte": "Referencia", }, inplace=True)
bd_negocio_DISTR.rename(columns={"ID": "Id", "TipoPublicacion(Clasica,Premium)": "Tipo de Publicación", "Att_SellerSKU": "SKU", "NumeroParte": "Referencia", }, inplace=True)
bd_negocio_DELATSA.rename(columns={"ID": "Id", "TipoPublicacion(Clasica,Premium)": "Tipo de Publicación", "Att_SellerSKU": "SKU", "NumeroParte": "Referencia", }, inplace=True)

In [31]:
#Transformacion
#Agregar "MLM" antes de cada valor en la columna "Id" para normalizar los valores
negocio_bd_GR8["Id"] = "MLM" + negocio_bd_GR8["Id"].astype(str)
bd_negocio_DISTR["Id"] = "MLM" + bd_negocio_DISTR["Id"].astype(str)
bd_negocio_DELATSA["Id"] = "MLM" + bd_negocio_DELATSA["Id"].astype(str)

In [32]:
#Transformacion
# Eliminar las filas donde el valor de la columna "Canal" sea "Mshops" ya que solo repite la información de los productos de Mercado Libre
negocio_bd_GR8 = negocio_bd_GR8[negocio_bd_GR8["Canal"] != "Mshops"]
bd_negocio_DISTR = bd_negocio_DISTR[bd_negocio_DISTR["Canal"] != "Mshops"]
bd_negocio_DELATSA = bd_negocio_DELATSA[bd_negocio_DELATSA["Canal"] != "Mshops"]

In [33]:
#Transformacion
# Convertir la columna 'Precio' a numérico en bd_negocio
negocio_bd_GR8['Precio'] = pd.to_numeric(negocio_bd_GR8['Precio'], errors='coerce')
bd_negocio_DISTR['Precio'] = pd.to_numeric(bd_negocio_DISTR['Precio'], errors='coerce')
bd_negocio_DELATSA['Precio'] = pd.to_numeric(bd_negocio_DELATSA['Precio'], errors='coerce')

In [34]:
#Limpiar
#eliminar productos sin precio
negocio_bd_GR8 = negocio_bd_GR8.dropna(subset=['Precio'])
bd_negocio_DISTR = bd_negocio_DISTR.dropna(subset=['Precio'])
bd_negocio_DELATSA = bd_negocio_DELATSA.dropna(subset=['Precio'])

In [35]:
#Transformación
#Traemos las columnas con las que queremos trabajar
negocio_bd_GR8 = negocio_bd_GR8[['Id', 'Titulo', 'Precio', 'Tipo de Publicación', 'SKU', 'Referencia', 'Canal']].copy()
bd_negocio_DISTR = bd_negocio_DISTR[['Id', 'Titulo', 'Precio', 'Tipo de Publicación', 'SKU', 'Referencia', 'Canal']].copy()
bd_negocio_DELATSA = bd_negocio_DELATSA[['Id', 'Titulo', 'Precio', 'Tipo de Publicación', 'SKU', 'Referencia', 'Canal']].copy()

In [36]:
negocio_bd = pd.concat([negocio_bd_GR8, bd_negocio_DISTR, bd_negocio_DELATSA], ignore_index=True)

Continuamos con DataFrame de 'compe_1' y 'compe_2'

In [38]:
#Transformación
#Traemos los productos que están como activos del día 2 que es con los precios que comparamos con los del negocio
compe_2 = compe_2[compe_2['Estado'] == 'Activa']

Eliminiamos filas que no contegan datos

In [39]:
#Limpiar
# Eliminar filas con NaN en columnas específicas
compe_1 = compe_1.dropna(subset=['Precio', 'Precio Oferta ML'])
compe_2 = compe_2.dropna(subset=['Precio', 'Precio Oferta ML'])


Creamos una columna en la que nos enseñe el precio actual de la publicación. Por que puede que su precio esté en la columna de 'Precio' o de 'Precio Oferta ML' dependiendo si tiene decuento

In [40]:
#Transformacion
# Ejemplo de código para DataFrame `compe_1`
compe_1['Precio_Final'] = compe_1.apply(
    lambda row: row['Precio Oferta ML'] if pd.notnull(row['Precio Oferta ML']) and row['Precio Oferta ML'] > 0 else row['Precio'],
    axis=1
)

# Ejemplo de código para DataFrame `compe_2`
compe_2['Precio_Final'] = compe_2.apply(
    lambda row: row['Precio Oferta ML'] if pd.notnull(row['Precio Oferta ML']) and row['Precio Oferta ML'] > 0 else row['Precio'],
    axis=1
)

Traemos las columnas con las que vamos a Trabajar

In [41]:
#Transformación
#Traemos las columnas con las que queremos trabajar
compe_1 = compe_1[['Id', 'Vendedor', 'Título', 'Precio_Final', 'Estado']].copy()
compe_2 = compe_2[['Id', 'Vendedor', 'Título', 'Precio_Final', 'Estado']].copy()

Seguimos con DataFrame de 'referencias'

Cambiamos el nombre de columnas

In [42]:
#Transformacion
#Cambiamnos el nombre de 'Id publicacion ' a 'Id' para vincularlo con los datasets del archivo de bd_competencia
ref.rename(columns={"Id publicacion ": "Id"}, inplace=True)

Traemos las columnas con las que vamos a Trabajar

In [43]:
##Transformacion
#Traemos las columnas con las que vamos a Trabajar
ref = ref[['Id', 'Titulo', 'Referencia', 'SKU', 'Empresa', 'Precio final']].copy()

Eliminamos las referencias que contengan "0", "Por el titulo" o "Por modelo" envés de un código válido

In [44]:
#Limpiar
# Filtrar las filas donde la columna 'Referencia' no contenga 0, "Por el titulo" o "Por modelo"
ref = ref.loc[~ref['Referencia'].isin(["0", "Por el titulo", "Por modelo"])]

Terminamos con DataFrame 'prom'

In [45]:
prom.rename(columns={"NOMBRE": "SKU"}, inplace=True)

Unión de datasets para comparar precios

Unimos los dataframes de la competencia con 'ref' para asignarles su código de Referencia

In [46]:
#Unir 'compe_1' con 'ref' en base a la columna 'Id' y agregar la columna 'Referencia'
#Unir 'compe_2' con 'ref' en base a la columna 'Id' y agregar la columna 'Referencia'
union_1_ref = compe_1.merge(ref[['Id', 'Referencia']], on='Id', how='left')
union_2_ref = compe_2.merge(ref[['Id', 'Referencia']], on='Id', how='left')

Los Id que no se les asignó código de referencia por no tener se eliminan

In [47]:
# Eliminar filas solo si tienen NaN en la columna 'Referencia'
union_1_ref = union_1_ref.dropna(subset=['Referencia'])
union_2_ref = union_2_ref.dropna(subset=['Referencia'])

Juntar las referencias duplicadas y solo entregar las que tienen menor precio

In [48]:
# Día 1: Obtener el precio mínimo de cada referencia
compe_1_listo = union_1_ref.loc[union_1_ref.groupby('Referencia')['Precio_Final'].idxmin()]

# Día 2: Obtener el precio mínimo de cada referencia
compe_2_listo = union_2_ref.loc[union_2_ref.groupby('Referencia')['Precio_Final'].idxmin()]

Comenzamos los filtros 

**Primer Filtro: Cambió el precio de la competencia de un día a otro?

Renombramos columnas para comparar un día con otro

In [49]:
# Renombramos la columna 'Precio' en compe_1 y compe_2 para indicar los días
compe_11 = compe_1_listo.rename(columns={'Precio_Final': 'Precio_Dia1'})
compe_22 = compe_2_listo.rename(columns={'Precio_Final': 'Precio_Dia2'})


Unimos dataframe de compe_11 y compe_22 para comparar precios y dar como resultado si cambió el precio de un dia a otro o no

In [50]:
# Unimos los DataFrames en base al Id para comparar precios del día 1 y el día 2
compe_compa = compe_11.merge(compe_22[['Referencia', 'Precio_Dia2']], on='Referencia', how='inner')

# Creamos una columna para identificar si el precio cambió
compe_compa['Cambio_Precio'] = compe_compa['Precio_Dia1'] != compe_compa['Precio_Dia2']

(Revisión)

In [51]:
compe_compa = compe_compa[compe_compa['Cambio_Precio'] == True]

**Segundo FIltro: Que precios tienen el negocio mayor que la competenca? Tomando en cuenta que la competencia cambiara sus precios de un dia a otro

1. Creamos un dataset con los datos de los código de referencia que sí cambiaron su oprecio de un día para otro

2. Unimos los dataframes de bd_negocio con el nuevo de precios true con la info de los cambios de precio

3. Hacemos la comparativa si nuestros precios son mayores que los de la competencia

In [52]:
# Filtrar precios_comparacion para obtener solo los registros donde el precio cambió
precios_true = compe_compa[compe_compa['Cambio_Precio']]

# Unir con negocio_bd en base a la columna 'Referencia'
compe_vs_negocio = precios_true.merge(negocio_bd[['Referencia', 'Precio', 'Id', 'Canal', 'SKU']], on='Referencia', how='inner')

# Crear una columna para verificar si el precio del negocio es mayor que el de la competencia (Precio_Dia2)
compe_vs_negocio['Negocio_Mayor_Competencia'] = compe_vs_negocio['Precio'] > compe_vs_negocio['Precio_Dia2']


Filtramos los casos en los que el precio del negocio es mayor que el de la competencia

In [53]:
# Filtrar solo los casos donde el precio del negocio es mayor
mayores_negocio = compe_vs_negocio[compe_vs_negocio['Negocio_Mayor_Competencia']]


Traemos las columnas que queremos al nuevo dataframe

In [54]:
mayores_negocio = mayores_negocio[['Id_x', 'Vendedor', 'Título', 'Precio_Dia2', 'Precio', 'Id_y', 'Canal', 'SKU', 'Negocio_Mayor_Competencia']].copy()

In [55]:
mayores_negocio.rename(columns={"Id_x": "Id_comp", "Precio_Dia2": "Precio_comp", "Precio": "Precio_nuestro", "Id_y": "Id_nuestro" }, inplace=True)

Creamos una nueva columna donde pondremos el nuevo precio sugerido de los productos que complieron con los filtros anteriores

**Tercer Filtro: Multiplicar productos x .99 del valor de la competencia

In [56]:
# Calcular el nuevo precio sugerido multiplicando por 0.99
mayores_negocio['Precio_Sugerido'] = mayores_negocio['Precio_comp'] * 0.99



Agregamos columna con precio desglosado de comisiones de mercado libre

In [57]:
# Calcular únicamente la comisión total de Mercado Libre
mayores_negocio['Comision_MercadoLibre'] = (mayores_negocio['Precio_Sugerido'] * (iva * isr - 1)) + envio

Agregar costo promedio del producto

Agregamos la utilidad del producto con el nuevo precio sugerido

In [58]:
# Realizar un merge en base a la columna SKU
r_precio_mayor = pd.merge(mayores_negocio, prom, on='SKU', how='left')

# Calcular la utilidad
r_precio_mayor['Utilidad_en_pesos'] = r_precio_mayor['Precio_Sugerido'] - r_precio_mayor['Comision_MercadoLibre'] - r_precio_mayor['COSTO_PROM']

# Opcional: Calcular el porcentaje de utilidad sobre el precio final
r_precio_mayor['Utilidad_Porcentaje'] = (r_precio_mayor['Utilidad_en_pesos'] / r_precio_mayor['Precio_Sugerido']) * 100


In [59]:


r_precio_mayor

,Id_comp,Vendedor,Título,Precio_comp,Precio_nuestro,Id_nuestro,Canal,SKU,Negocio_Mayor_Competencia,Precio_Sugerido,Comision_MercadoLibre,COSTO_PROM,Utilidad_en_pesos,Utilidad_Porcentaje
0,MLM2681736612,CNFUBBMXF,"Alternador Volkswagen Vw Vocho, Beetle, Sedán,...",1285.46,1401.0,MLM1938567861,MercadoLibre,ALT-B13080,True,1272.6054,485.525758,687.062,100.017642,7.859282
1,MLM2681736612,CNFUBBMXF,"Alternador Volkswagen Vw Vocho, Beetle, Sedán,...",1285.46,1401.0,MLM1938568155,MercadoLibre,ALT-B13080,True,1272.6054,485.525758,687.062,100.017642,7.859282
2,MLM2681736612,CNFUBBMXF,"Alternador Volkswagen Vw Vocho, Beetle, Sedán,...",1285.46,1401.0,MLM1938568183,MercadoLibre,ALT-B13080,True,1272.6054,485.525758,687.062,100.017642,7.859282
3,MLM2681736612,CNFUBBMXF,"Alternador Volkswagen Vw Vocho, Beetle, Sedán,...",1285.46,1401.0,MLM2011473197,MercadoLibre,ALT-B13080,True,1272.6054,485.525758,687.062,100.017642,7.859282
4,MLM2681736612,CNFUBBMXF,"Alternador Volkswagen Vw Vocho, Beetle, Sedán,...",1285.46,1401.0,MLM2440224770,MercadoLibre,ALT-B13080,True,1272.6054,485.525758,687.062,100.017642,7.859282
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103,MLM2706696058,CNFUBBMXF,Marcha Para Ford Courier Fiesta 2001-2012 Ikon...,1389.21,1805.0,MLM887050320,MercadoLibre,MAR-V438180,True,1375.3179,517.448803,656.092,201.777097,14.671306
104,MLM2706696058,CNFUBBMXF,Marcha Para Ford Courier Fiesta 2001-2012 Ikon...,1389.21,1805.0,MLM887050323,MercadoLibre,MAR-V438180,True,1375.3179,517.448803,656.092,201.777097,14.671306
105,MLM2706696058,CNFUBBMXF,Marcha Para Ford Courier Fiesta 2001-2012 Ikon...,1389.21,1805.0,MLM887050326,MercadoLibre,MAR-V438180,True,1375.3179,517.448803,656.092,201.777097,14.671306
106,MLM2706696058,CNFUBBMXF,Marcha Para Ford Courier Fiesta 2001-2012 Ikon...,1389.21,1805.0,MLM2011382713,Ambos,MAR-V438180,True,1375.3179,517.448803,656.092,201.777097,14.671306


Agregamos otro DataFrame con un nuevo formato que será el formato Requerido por Delatsa

In [60]:
r_precio_mayor_formato = r_precio_mayor[['Id_nuestro', 'Precio_Sugerido', 'Precio_nuestro' ]].copy()
r_precio_mayor_formato['Oferta_Revisada'] = None
r_precio_mayor_formato['Precio_Exclusivo_Meli_'] = " " 

#Movemos la columna 'Oferta_Revisada' de lugar para que quede como el formato que nos pidió Delatsa
columna_movers = r_precio_mayor_formato.pop('Oferta_Revisada')
r_precio_mayor_formato.insert(2, 'Oferta_Revisada', columna_movers)
r_precio_mayor_formato['Precio_Exclusivo_Meli_'] = " " 

In [61]:
r_precio_mayor_formato

,Id_nuestro,Precio_Sugerido,Oferta_Revisada,Precio_nuestro,Precio_Exclusivo_Meli_
0,MLM1938567861,1272.6054,None,1401.0,
1,MLM1938568155,1272.6054,None,1401.0,
2,MLM1938568183,1272.6054,None,1401.0,
3,MLM2011473197,1272.6054,None,1401.0,
4,MLM2440224770,1272.6054,None,1401.0,
...,...,...,...,...,...
103,MLM887050320,1375.3179,None,1805.0,
104,MLM887050323,1375.3179,None,1805.0,
105,MLM887050326,1375.3179,None,1805.0,
106,MLM2011382713,1375.3179,None,1805.0,


Filtramos por si salieron valores negativos, que significan que estaríamos perdiendo dinero si cambiaramos el precio

In [62]:
# Filtrar el dataframe para que solo incluya filas donde 'Utilidad' es mayor o igual a cero
r_precio_mayor_filtrado = r_precio_mayor[r_precio_mayor['Utilidad_en_pesos'] >= 0]

In [63]:
r_precio_mayor_filtrado


,Id_comp,Vendedor,Título,Precio_comp,Precio_nuestro,Id_nuestro,Canal,SKU,Negocio_Mayor_Competencia,Precio_Sugerido,Comision_MercadoLibre,COSTO_PROM,Utilidad_en_pesos,Utilidad_Porcentaje
0,MLM2681736612,CNFUBBMXF,"Alternador Volkswagen Vw Vocho, Beetle, Sedán,...",1285.46,1401.0,MLM1938567861,MercadoLibre,ALT-B13080,True,1272.6054,485.525758,687.062,100.017642,7.859282
1,MLM2681736612,CNFUBBMXF,"Alternador Volkswagen Vw Vocho, Beetle, Sedán,...",1285.46,1401.0,MLM1938568155,MercadoLibre,ALT-B13080,True,1272.6054,485.525758,687.062,100.017642,7.859282
2,MLM2681736612,CNFUBBMXF,"Alternador Volkswagen Vw Vocho, Beetle, Sedán,...",1285.46,1401.0,MLM1938568183,MercadoLibre,ALT-B13080,True,1272.6054,485.525758,687.062,100.017642,7.859282
3,MLM2681736612,CNFUBBMXF,"Alternador Volkswagen Vw Vocho, Beetle, Sedán,...",1285.46,1401.0,MLM2011473197,MercadoLibre,ALT-B13080,True,1272.6054,485.525758,687.062,100.017642,7.859282
4,MLM2681736612,CNFUBBMXF,"Alternador Volkswagen Vw Vocho, Beetle, Sedán,...",1285.46,1401.0,MLM2440224770,MercadoLibre,ALT-B13080,True,1272.6054,485.525758,687.062,100.017642,7.859282
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102,MLM2706696058,CNFUBBMXF,Marcha Para Ford Courier Fiesta 2001-2012 Ikon...,1389.21,1805.0,MLM887050317,MercadoLibre,MAR-V438180,True,1375.3179,517.448803,656.092,201.777097,14.671306
103,MLM2706696058,CNFUBBMXF,Marcha Para Ford Courier Fiesta 2001-2012 Ikon...,1389.21,1805.0,MLM887050320,MercadoLibre,MAR-V438180,True,1375.3179,517.448803,656.092,201.777097,14.671306
104,MLM2706696058,CNFUBBMXF,Marcha Para Ford Courier Fiesta 2001-2012 Ikon...,1389.21,1805.0,MLM887050323,MercadoLibre,MAR-V438180,True,1375.3179,517.448803,656.092,201.777097,14.671306
105,MLM2706696058,CNFUBBMXF,Marcha Para Ford Courier Fiesta 2001-2012 Ikon...,1389.21,1805.0,MLM887050326,MercadoLibre,MAR-V438180,True,1375.3179,517.448803,656.092,201.777097,14.671306


Sacamos la utilidad del costo del producto con el nuevo precio sugerido

Filtros 2


In [64]:
# Filtrar compe_compa para obtener solo los registros donde el precio cambió de un día a otro
precios_true_2 = compe_compa[compe_compa['Cambio_Precio']]

# Unir precios_true con negocio_bd en base a la columna 'Referencia'
compe_vs_negocio_2 = precios_true_2.merge(negocio_bd[['Referencia', 'Precio', 'Id', 'Canal', 'SKU']], on='Referencia', how='inner')

# Crear una columna para verificar si el precio del negocio es menor que el de la competencia (Precio_Dia2)
compe_vs_negocio_2['Negocio_Menor_Competencia'] = compe_vs_negocio_2['Precio'] < compe_vs_negocio_2['Precio_Dia2']

# Filtrar solo los casos donde el precio del negocio es menor
menores_negocio = compe_vs_negocio_2[compe_vs_negocio_2['Negocio_Menor_Competencia']]


In [65]:
menores_negocio.rename(columns={"Id_x": "Id_comp", "Precio_Dia2": "Precio_comp", "Precio": "Precio_nuestro", "Id_y": "Id_nuestro" }, inplace=True)

C:\Users\dell\AppData\Local\Temp\ipykernel_10620\1802238037.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  menores_negocio.rename(columns={"Id_x": "Id_comp", "Precio_Dia2": "Precio_comp", "Precio": "Precio_nuestro", "Id_y": "Id_nuestro" }, inplace=True)


In [66]:
# Calcular el nuevo precio sugerido multiplicando por 0.99
menores_negocio['Precio_Sugerido'] = menores_negocio['Precio_comp'] * 0.99

C:\Users\dell\AppData\Local\Temp\ipykernel_10620\2882090104.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  menores_negocio['Precio_Sugerido'] = menores_negocio['Precio_comp'] * 0.99


Agregar columna con desglose de mercado libre comisiones


In [67]:
# Calcular únicamente la comisión total de Mercado Libre
menores_negocio['Comision_MercadoLibre'] = (menores_negocio['Precio_Sugerido'] * (iva * isr - 1)) + envio

C:\Users\dell\AppData\Local\Temp\ipykernel_10620\1076637241.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  menores_negocio['Comision_MercadoLibre'] = (menores_negocio['Precio_Sugerido'] * (iva * isr - 1)) + envio


In [68]:
menores_negocio

,Id_comp,Vendedor,Título,Precio_Dia1,Estado,Referencia,Precio_comp,Cambio_Precio,Precio_nuestro,Id_nuestro,Canal,SKU,Negocio_Menor_Competencia,Precio_Sugerido,Comision_MercadoLibre
4,MLM2681736612,CNFUBBMXF,"Alternador Volkswagen Vw Vocho, Beetle, Sedán,...",110.00,Pausada,13080,1285.46,True,1190.85,MLM2016168873,MercadoLibre,ALT-B13080,True,1272.6054,485.525758
37,MLM1866875061,CIOSA AUTOPARTES,Alternador Cr-v 2.4 2002 2003 2004 2005 2006,2948.19,Activa,13966,2963.00,True,2234.00,MLM3284031160,MercadoLibre,ALT-M13966,True,2933.3700,1001.691396
38,MLM1866875061,CIOSA AUTOPARTES,Alternador Cr-v 2.4 2002 2003 2004 2005 2006,2948.19,Activa,13966,2963.00,True,2234.00,MLM3284108880,MercadoLibre,ALT-M13966,True,2933.3700,1001.691396


Agregamos la utilidad del producto con el nuevo precio sugerido

In [69]:
# Realizar un merge en base a la columna SKU
r_precio_menor = pd.merge(menores_negocio, prom, on='SKU', how='left')

# Calcular la utilidad
r_precio_menor['Utilidad_en_pesos'] = r_precio_menor['Precio_Sugerido'] - r_precio_menor['Comision_MercadoLibre'] - r_precio_menor['COSTO_PROM']

# Opcional: Calcular el porcentaje de utilidad sobre el precio final
r_precio_menor['Utilidad_Porcentaje'] = (r_precio_menor['Utilidad_en_pesos'] / r_precio_menor['Precio_Sugerido']) * 100

Agregamos otro DataFrame con un nuevo formato que será el formato Requerido por Delatsa

In [70]:
r_precio_menor_formato = r_precio_menor[['Id_nuestro', 'Precio_Sugerido', 'Precio_nuestro' ]].copy()
r_precio_menor_formato['Oferta_Revisada'] = None
r_precio_menor_formato['Precio_Exclusivo_Meli_'] = " " 

In [71]:
columna_mover = r_precio_menor_formato.pop('Oferta_Revisada')
r_precio_menor_formato.insert(2, 'Oferta_Revisada', columna_mover)
r_precio_menor_formato['Precio_Exclusivo_Meli_'] = " " 

In [72]:
r_precio_menor_formato

,Id_nuestro,Precio_Sugerido,Oferta_Revisada,Precio_nuestro,Precio_Exclusivo_Meli_
0,MLM2016168873,1272.6054,None,1190.85,
1,MLM3284031160,2933.3700,None,2234.00,
2,MLM3284108880,2933.3700,None,2234.00,


Sacamos la utilidad del costo

In [73]:
# Filtrar el dataframe para que solo incluya filas donde 'Utilidad' es mayor o igual a cero
r_precio_menor_filtrado = r_precio_menor[r_precio_menor['Utilidad_en_pesos'] >= 0]

In [74]:
r_precio_menor

,Id_comp,Vendedor,Título,Precio_Dia1,Estado,Referencia,Precio_comp,Cambio_Precio,Precio_nuestro,Id_nuestro,Canal,SKU,Negocio_Menor_Competencia,Precio_Sugerido,Comision_MercadoLibre,COSTO_PROM,Utilidad_en_pesos,Utilidad_Porcentaje
0,MLM2681736612,CNFUBBMXF,"Alternador Volkswagen Vw Vocho, Beetle, Sedán,...",110.00,Pausada,13080,1285.46,True,1190.85,MLM2016168873,MercadoLibre,ALT-B13080,True,1272.6054,485.525758,687.062,100.017642,7.859282
1,MLM1866875061,CIOSA AUTOPARTES,Alternador Cr-v 2.4 2002 2003 2004 2005 2006,2948.19,Activa,13966,2963.00,True,2234.00,MLM3284031160,MercadoLibre,ALT-M13966,True,2933.3700,1001.691396,959.266,972.412604,33.150015
2,MLM1866875061,CIOSA AUTOPARTES,Alternador Cr-v 2.4 2002 2003 2004 2005 2006,2948.19,Activa,13966,2963.00,True,2234.00,MLM3284108880,MercadoLibre,ALT-M13966,True,2933.3700,1001.691396,959.266,972.412604,33.150015


Archivo con formato solicitado por Delatsa

In [75]:
file = 'resultados_formato_delatsa.xlsx'

with pd.ExcelWriter(file, engine='openpyxl', mode='w') as writer:
    r_precio_mayor_formato.to_excel(writer, sheet_name='precios_mayores', index=False)

    r_precio_menor_formato.to_excel(writer, sheet_name='precios_menores', index=False)

print("Archivo Excel creado o actualizado con éxito.")

Archivo Excel creado o actualizado con éxito.


Archivo 'resultados' con los Resultados en Excel

In [76]:
# Ruta donde se guardará el archivo Excel (reemplazando si ya existe)
file = 'resultados.xlsx'

# Crear el archivo Excel nuevo o sobrescribir el existente
with pd.ExcelWriter(file, engine='openpyxl', mode='w') as writer:
    # Exportar resultados de 'mayores_negocio' a la primera hoja (reemplaza si existe)
    r_precio_mayor.to_excel(writer, sheet_name='mayor_precio', index=False)
    
    # Exportar resultados de 'r_precio_menor' a la segunda hoja (reemplaza si existe)
    r_precio_menor.to_excel(writer, sheet_name='menor_precio', index=False)

print("Archivo Excel creado o actualizado con éxito.")

Archivo Excel creado o actualizado con éxito.


Archivo 'cambio_de_precio' para cambiar precio

In [77]:
# Ruta donde se guardará el archivo Excel (reemplazando si ya existe)
file_path = 'cambio_de_precio.xlsx'

# Crear el archivo Excel nuevo o sobrescribir el existente
with pd.ExcelWriter(file_path, engine='openpyxl', mode='w') as writer:
    # Exportar la columna 'Id_nuestro' de 'mayores_negocio' a la primera hoja (reemplaza si existe)
    r_precio_mayor_filtrado[['Id_nuestro', 'SKU']].to_excel(writer, sheet_name='mayor_precio', index=False)
    
    # Exportar la columna 'Id_nuestro' de 'menores_negocio' a la segunda hoja (reemplaza si existe)
    r_precio_menor_filtrado[['Id_nuestro', 'SKU']].to_excel(writer, sheet_name='menor_precio', index=False)

print("Archivo Excel creado o actualizado con éxito.")

Archivo Excel creado o actualizado con éxito.
